In [63]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as soup
import pandas as pd
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import re

In [27]:
def get_todays_date():
    now = datetime.now()
    now = datetiem(now.year, now.month, now.day)
    return now

In [23]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
#options.add_argument('--incognito')
#options.add_argument('--headless')
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features")
options.add_argument('headless')
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument('--disable-dev-shm-usage')
# options.binary_location('/opt/google/chrome/google-chrome')

browser = webdriver.Chrome("chromedriver", options=options)

# IE_path = 'D:\selds\IEDriver\IEDriverServer'
# options = webdriver.IeOptions()
# browser = webdriver.Ie(IE_path, options=options)

In [64]:
def extract_data(page_source):
    df_c =  pd.DataFrame(columns = ['name','earned', 'apr', 'liquidity', 'multiplier'])
    
    page_soup = soup(page_source, "html.parser")
    table = page_soup.findAll("table")#, {"class": "sc-hRMXGO hbbpxV"})
    #heads = table[0].thead.tr.findAll("th")
    rows =  table[0].tbody.findAll("tr")
    for row in rows: 
        divs = row.findAll("div", {"color": "text"})
        name = divs[0].text
        #name
        earned = row.find_all("span")[0].text
        #earned
        apr = row.find_all(text=re.compile('%'))[0]
        #apr
        liq = divs[1].text
        #liq
        mul = row.find_all(text=re.compile('x'))[0]
        #mul
        #print(f'{name} {earned} {apr} {liq} {mul}')
        df_c.at[len(df_c)] = [name,earned,apr,liq,mul]
    return df_c

In [65]:
df = extract_data(page_source)

IndexError: list index out of range

### pulling farm data:

In [28]:
farm_payment = 'CAKE'

where = 'PancakeSwap'

In [25]:
time.sleep(5)
browser.get("https://pancakeswap.finance/farms")
prev_len = 0
curr_len = 100
count = 0
while count<5:
    print(curr_len)
    time.sleep(3)
    page_source = browser.page_source
    #df = extract_data(page_source)
    time.sleep(3)
    #print(df)
    body = browser.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    body.send_keys(Keys.PAGE_DOWN)
    curr_len = len(page_source)
    if curr_len == prev_len:
        count = count+1
    else:
        count = 0
    prev_len = curr_len

100
63341
79380
95353
111339
127286
143266
159245
159559
175493
191452
202130
202130
202130
202130
202130


In [38]:
page_source = browser.page_source
df = extract_data(page_source)
df['date'] = get_todays_date()
df['farm_payment'] = farm_payment
df['where'] = where

[<table class="sc-fTQuIj fxTeMH"><tbody class="sc-bUhGej iTsbA-d"><tr class="sc-DtlDN jbJFqh"><td><div class="sc-iukwUI bgUaxT"><div><div class="sc-hGnhcH hSbSo"><div class="sc-dkQkSb fTqHaS"><div class="sc-iwjezw gGiThJ sc-gGPAug cJEnKB" height="40" width="40"><img alt="icon" class="sc-cxpRKc hxgtkf" src="/images/farms/cake-bnb.svg"/></div><div><div class="sc-gsDJrp dzNkWZ" color="text">CAKE-BNB</div></div></div></div></div></div></td><td><div class="sc-iukwUI bgUaxT"><div><div class="sc-ehIIUG iBvdXq">Earned</div><div class="sc-hGnhcH hSbSo"><span class="sc-jivCuG hfCxiQ">0</span></div></div></div></td><td><div class="sc-iukwUI bgUaxT"><div><div class="sc-ehIIUG iBvdXq">APR</div><div class="sc-hGnhcH hSbSo"><div class="sc-cHzrye dBOkBe"><div class="sc-JkiRB hHABJO">71.24%</div></div></div></div></div></td><td><div class="sc-iukwUI bgUaxT"><div><div class="sc-hGnhcH hSbSo"><div class="sc-hkgtOd jeFvJu"><svg class="sc-bdvvaa dqTYWn sc-bSqbRM giJmne" color="primary" viewbox="0 0 24 24" 

IndexError: list index out of range

In [ ]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()



def insert_pancakswap_data_df(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["Date"].timestamp()
        try:
            pair2 = row["Pair_2"] if row["Pair_2"] else ' '
            print(row["Date"], row["Pair_1"], pair2, row["Pool_rate"], row["pool_tvl"], row["Farm_payment"], row["Where"], row["includes_stablecoin"])
            sql = "INSERT INTO `pancakswap_farm_data` (`timestamp_dt`, `pair1`,`pair2`,`pool_rate`,`pool_size`,`farm_payment`,`where`,`includes_stablecoin`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, row["Pair_1"], pair2, row["Pool_rate"], row["pool_tvl"], row["Farm_payment"], row["Where"], row["includes_stablecoin"]))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()

In [6]:
df.to_csv('Pancakeswap_Farms.csv',index=None)

In [7]:
browser.close()

In [ ]:
df[]